In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Ejemplo de Walk-Forward Optimization
import yfinance as yf

In [2]:
df_original = yf.Ticker('NQ=F')
df_original = df_original.history(start='2010-01-01', end='2024-01-01')

### Función de backtest para cada combinación de parámetros


In [ ]:
def backtest_strategy(data,sma_1,sma_2):
    df = data.copy()